### Import Libraries

In [42]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import warnings
import pandas as pd
import numpy as np
import os

warnings.filterwarnings('ignore')

### Website - https://www.mims.com/

In [6]:
# 🔐 Your login credentials (replace with your actual username/password)
EMAIL = ""
PASSWORD = ""


# Target URL
# base_url = "https://www.mims.com"
login_url = "https://www.mims.com/login"  # Login page
# search_url = "https://www.mims.com/india/drug/search?q=Antacids%2C+Antireflux+Agents+%26+Antiulcerants&mtype=brand"
# search_url = "https://www.mims.com/india/drug/search?q=GIT+Regulators%2C+Antiflatulents+%26+Anti-Inflammatories&mtype=brand"
# search_url = "https://www.mims.com/india/drug/search?q=cardiac+drugs&mtype=brand&code=2a"
# search_url = "https://www.mims.com/india/drug/search?q=Calcium+Antagonists&mtype=brand"
search_url = "https://www.mims.com/india/drug/search?q=Beta-Blockers&mtype=brand"
output_dir = "mims_medicine_data/05/"

# Page counter
page_number = 1

# while page_number <= 16:
while True:
    base_url = "https://www.mims.com/"
    # Setup Selenium WebDriver
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)  # Keeps the browser open
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(base_url)
    
    # 🔐 Log in to the website
    try:
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "Email")))
        # button = driver.find_element(By.CLASS_NAME, "btn a_signin")
        button = driver.find_element(By.XPATH, "//a[contains(@href, '/account/login?returnurl=')]")
        button.click()
        time.sleep(3)
        driver.find_element(By.ID, "Input_Email").send_keys(EMAIL)
        driver.find_element(By.ID, "Input_Password").send_keys(PASSWORD)
        driver.find_element(By.NAME, "Input.Button").click()
        print("✅ Logged in successfully!")
    
        # Wait for login to complete
        time.sleep(5)
    
        # Go to the search results page
        driver.get(search_url)
    
    except Exception as e:
        print(f"❌ Login failed: {e}")
        driver.quit()
        exit()
    
    # Target URL
    # search_url = "https://www.mims.com/india/drug/search?q=Antacids%2C+Antireflux+Agents+%26+Antiulcerants&mtype=brand"
    base_url1 = search_url
    
    # html_content = driver.find_element(By.TAG_NAME, "body").get_attribute("outerHTML")
    # print(html_content)
    
    # 📥 Extract medicine details from multiple pages
    all_medicines = []
    
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    
    # Wait for the pagination to load and find the last page number
    try:
        last_page_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'page=')][last()]"))
        )
        last_page_num = int(last_page_element.get_attribute("href").split("page=")[-1])
    except:
        print("⚠️ Could not determine the total number of pages. Defaulting to 1.")
        last_page_num = 1
    
    # Loop through all pages
    for page in range(page_number, last_page_num + 1):
        page_url = f"{base_url1}&page={page}"
        driver.get(page_url)
        
        print(f"📄 Scraping Page {page}...")
    
        try:
            # Wait for the medicine list to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'search-border-style')]"))
            )
    
            # Extract all medicine cards
            medicine_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'search-border-style')]")
    
            # Store medicine data for this page
            medicines_this_page = []
    
            for card in medicine_cards:
                try:
                    # Extract medicine name
                    name = card.find_element(By.XPATH, ".//a/h1").text
    
                    # Extract generic name - Handle both cases
                    try:
                        generic_name = card.find_element(By.XPATH, ".//span[@class='class-text-red']/a").text  # Case 1
                    except:
                        try:
                            generic_name = card.find_element(By.XPATH, ".//span[@class='class-text header']").text  # Case 2
                        except:
                            generic_name = "N/A"  # If neither case is found
    
                    medicines_this_page.append(f"{name} - {generic_name}")
                    print(f"✅ Page {page}: {name} - {generic_name}")
    
                except Exception as e:
                    print(f"⚠️ Skipping a card due to missing data: {e}")
    
            # Save results for this page
            filename = f"{output_dir}/medicines_page_{page}.txt"
            with open(filename, "w", encoding="utf-8") as f:
                f.write("\n".join(medicines_this_page))
    
            print(f"📄 Data saved for Page {page} -> {filename}")
    
        except Exception as e:
            print(f"⚠️ Error on page {page}: {e}")
            break
    
    # Close the browser
    driver.quit()
    page_number = page
    print("✅ All pages scraped successfully!")
    print("Starting with page number", page_number)
    time.sleep(10)
    if page_number == last_page_num:
        break

✅ Logged in successfully!
📄 Scraping Page 1...
✅ Page 1: Aamin-A - Amlodipine + Atenolol
✅ Page 1: AB-Dip-AT - Amlodipine + Atenolol
✅ Page 1: Abiten - Atenolol
✅ Page 1: Abiten-A - Amlodipine + Atenolol
✅ Page 1: Abizox-P - alprazolam + propranolol
✅ Page 1: AB-Lol - Labetalol
✅ Page 1: Acard - Amlodipine + Atenolol
✅ Page 1: A-Card - Atenolol
✅ Page 1: A-Card-AM - Amlodipine + Atenolol
✅ Page 1: Ace Revelol - metoprolol succinate + ramipril
📄 Data saved for Page 1 -> mims_medicine_data/05//medicines_page_1.txt
📄 Scraping Page 2...
✅ Page 2: Ace-Revelol Tab - metoprolol + ramipril
✅ Page 2: Acord-A - Amlodipine + Atenolol
✅ Page 2: Acord-M - amlodipine besylate + metoprolol succinate
✅ Page 2: Actiblok-AM - amlodipine + metoprolol
✅ Page 2: Actiblok-IPR - Metoprolol
✅ Page 2: Actocard - Metoprolol
✅ Page 2: Adbeta - S-atenolol
✅ Page 2: Adbeta-H - hydrochlorothiazide + S-atenolol
✅ Page 2: Adipin Plus - Amlodipine + Atenolol
✅ Page 2: Adorant - Amlodipine + Atenolol
📄 Data saved for P

### Website - https://cdsco.gov.in

In [16]:
import os
import time
import requests
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)  # Keeps the browser open
driver = webdriver.Chrome(service=service, options=options)

# Target website URL
base_url = "https://cdsco.gov.in"
url = "https://cdsco.gov.in/opencms/opencms/en/Approval_new/Approved-New-Drugs/"
driver.get(url)

# Create a folder to store PDFs
download_folder = "CDSCO_PDFs"
os.makedirs(download_folder, exist_ok=True)

def download_pdf(pdf_url, filename):
    """Downloads a PDF file from the given URL and saves it locally."""
    response = requests.get(pdf_url, stream=True)
    if response.status_code == 200:
        filepath = os.path.join(download_folder, filename)
        with open(filepath, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"✅ Downloaded: {filename}")
    else:
        print(f"❌ Failed to download: {filename}")

page_number = 1  # Track the current page

while True:
    try:
        print(f"🔍 Scraping page {page_number}...")

        # Wait for the page to load before extracting links
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))

        # Re-find JSP links on each iteration to avoid stale elements
        jsp_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'download_file_division.jsp?num_id=')]")

        if not jsp_links:
            print(f"⚠️ No JSP links found on page {page_number}.")

        # Extract the href values first to avoid stale elements
        jsp_hrefs = [urljoin(base_url, link.get_attribute("href")) for link in jsp_links]

        # Process each JSP link separately
        for i, jsp_url in enumerate(jsp_hrefs, start=1):
            driver.get(jsp_url)  # Open JSP page
            time.sleep(2)  # Allow page to load
            
            try:
                # Find the iframe that contains the actual PDF URL
                iframe = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.TAG_NAME, "iframe"))
                )
                relative_pdf_url = iframe.get_attribute("src")  # Extract PDF URL
                full_pdf_url = urljoin(base_url, relative_pdf_url)  # Convert to full URL

                # Generate a unique filename and download the PDF
                pdf_name = f"page{page_number}_document_{i}.pdf"
                download_pdf(full_pdf_url, pdf_name)

            except Exception as e:
                print(f"⚠️ Could not extract PDF URL from {jsp_url}: {e}")

            # Go back to the main page before processing the next link
            driver.get(url)
            time.sleep(3)  # Wait for the page to reload before finding new elements

        # Check if the "Next" button exists and click it
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "dt-paging-button.next"))
            )
            driver.execute_script("arguments[0].click();", next_button)  # JavaScript click
            time.sleep(3)  # Wait for the next page to load
            page_number += 1  # Increment page count

        except:
            print("✅ No more pages left. Exiting.")
            break  # Exit loop if no "Next" button

    except Exception as e:
        print(f"⚠️ Error on page {page_number}: {e}")
        break

# Close the browser
driver.quit()


🔍 Scraping page 1...
✅ Downloaded: page1_document_1.pdf
✅ Downloaded: page1_document_2.pdf
✅ Downloaded: page1_document_3.pdf
✅ Downloaded: page1_document_4.pdf
✅ Downloaded: page1_document_5.pdf
✅ Downloaded: page1_document_6.pdf
✅ Downloaded: page1_document_7.pdf
✅ Downloaded: page1_document_8.pdf
✅ Downloaded: page1_document_9.pdf
✅ Downloaded: page1_document_10.pdf
🔍 Scraping page 2...
✅ Downloaded: page2_document_1.pdf
✅ Downloaded: page2_document_2.pdf
✅ Downloaded: page2_document_3.pdf
✅ Downloaded: page2_document_4.pdf
✅ Downloaded: page2_document_5.pdf
✅ Downloaded: page2_document_6.pdf
✅ Downloaded: page2_document_7.pdf
✅ Downloaded: page2_document_8.pdf
✅ Downloaded: page2_document_9.pdf
✅ Downloaded: page2_document_10.pdf
🔍 Scraping page 3...
✅ Downloaded: page3_document_1.pdf
✅ Downloaded: page3_document_2.pdf
✅ Downloaded: page3_document_3.pdf
✅ Downloaded: page3_document_4.pdf
✅ Downloaded: page3_document_5.pdf
✅ Downloaded: page3_document_6.pdf
✅ Downloaded: page3_docum

KeyboardInterrupt: 

### Reading PDF Schema

Reading and parsing pdf files gave me some hard time as the parsing could not completely understand separating lines between two entries so I did not include this data in the databse.

In [ ]:
%%time
from tabula import read_pdf

all_drugs = os.listdir("./CDSCO_PDFs/")
if '.ipynb_checkpoints' in all_drugs:
    all_drugs.remove('.ipynb_checkpoints')
    
print(len(all_drugs))
dfs = {}
for i, file in enumerate(all_drugs):
    print(i, file)
    df1 = read_pdf(f'./CDSCO_PDFs/{file}',pages='all', stream=True)
    df2 = df1[0]
    if len(df2) > 1:
        for i in range(1,len(df1)):
            dfx = df1[i]
            dfx.loc[-1] = dfx.columns
            dfx.columns = df1[0].columns
            dfx = dfx.sort_index()
            df2 = pd.concat([df2, dfx])
    dfs[file] = df2.reset_index(drop=True)

### Creating Medicine Database

In [34]:
%%time

medicines = pd.DataFrame(columns=['name','generic_name'])
dirs = os.listdir("./mims_medicine_data/")
if '.ipynb_checkpoints' in dirs:
    dirs.remove('.ipynb_checkpoints')

for direc in dirs:
    files = os.listdir(f"./mims_medicine_data/{direc}")
    if '.ipynb_checkpoints' in files:
        files.remove('.ipynb_checkpoints')
    for file in files:
        # print(f"./mims_medicine_data/{direc}/{file}")
        f = open(f"./mims_medicine_data/{direc}/{file}")
        names = f.readlines()
        f.close()
        # print(names)
        df1 = pd.DataFrame(names)
        df1['name'] = df1[0].map(lambda x : x.split('-')[0].strip())
        df1['generic_name'] = df1[0].map(lambda x : x.split('-')[1].strip())
        df1 = df1.drop(0,axis=1)
        df1['direc'] = f"./mims_medicine_data/{direc}"
        medicines = pd.concat([medicines, df1], axis=0)

medicines

CPU times: user 333 ms, sys: 14.2 ms, total: 348 ms
Wall time: 346 ms


,name,generic_name,direc
0,Acylcar,levocarnitine,./mims_medicine_data/03
1,Adenocor,Adenosine,./mims_medicine_data/03
2,Adenoject,Adenosine,./mims_medicine_data/03
3,Adenoz,Adenosine,./mims_medicine_data/03
4,Adneon,Adenosine,./mims_medicine_data/03
...,...,...,...
5,Frecid,magaldrate + simethicone,./mims_medicine_data/01
6,Fudone,Famotidine,./mims_medicine_data/01
7,Fulpan,Pantoprazole,./mims_medicine_data/01
8,Fulpan,D,./mims_medicine_data/01


In [35]:
medicines.groupby('direc',as_index=False).size()

,direc,size
0,./mims_medicine_data/01,5200
1,./mims_medicine_data/02,407
2,./mims_medicine_data/03,159
3,./mims_medicine_data/04,801
4,./mims_medicine_data/05,982


In [44]:
medicines.to_parquet("./medicines.parquet")

### Similarity Matching

In [36]:
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

In [37]:
def findSimilarity(w1, w2):
    normalized_levenshtein = NormalizedLevenshtein()

    return normalized_levenshtein.similarity(w1, w2)

In [40]:
%%time

string = 'Fudon1'
all_simi = pd.DataFrame(columns=['input','match','score'])
for idx, row in medicines.iterrows():
    w1 = string
    for col in ['name','generic_name']:
        w2 = row[col]
        simi = findSimilarity(w1, w2)
        all_simi.loc[len(all_simi)] = [w1, w2, simi]

CPU times: user 4.43 s, sys: 41.2 ms, total: 4.47 s
Wall time: 4.45 s


In [47]:
for match in all_simi.sort_values('score',ascending=False).head(3)[['match','score']].values:
    print(match[0], np.round(match[1],3))

Fudone 0.833
Nodon 0.5
Fupan 0.5
